In [80]:
#import packages
import pandas as pd 
import numpy as np
import pandas_profiling
import pycountry as p
import pycountry_convert as pc
from pandas_profiling.utils.cache import cache_file
from sklearn.preprocessing import OneHotEncoder
import datetime as dt
pd.set_option('display.float_format', lambda x: '%.1f' % x)

In [81]:
data = pd.read_csv("loans.csv")

In [82]:
data.head()

,LOAN_ID,LOAN_NAME,ORIGINAL_LANGUAGE,DESCRIPTION,DESCRIPTION_TRANSLATED,FUNDED_AMOUNT,LOAN_AMOUNT,STATUS,IMAGE_ID,VIDEO_ID,...,LENDER_TERM,NUM_LENDERS_TOTAL,NUM_JOURNAL_ENTRIES,NUM_BULK_ENTRIES,TAGS,BORROWER_NAMES,BORROWER_GENDERS,BORROWER_PICTURED,REPAYMENT_INTERVAL,DISTRIBUTION_MODEL
0,822769,Ana Olaya,Spanish,En el cantón 24 De Mayo de la provincia de Ma...,"In the 24 De Mayo canton, Manabí province, kno...",825.0,825.0,funded,1772632.0,NaN,...,8.0,9,1,1,NaN,Ana Olaya,female,true,monthly,field_partner
1,758447,JOVELINE\t,English,Joveline is married and has two children. She ...,Joveline is married and has two children. She ...,525.0,525.0,funded,1677756.0,NaN,...,8.0,12,2,1,"#Animals, #Parent, #Woman-Owned Business",JOVELINE\t,female,true,monthly,field_partner
2,443481,JUAN ANGEL,Spanish,"Juan es un hombre de 54 años, es felizmente ca...","Juan, 54, is happily married. He lives with h...",400.0,400.0,funded,1127992.0,NaN,...,14.0,15,2,1,NaN,JUAN ANGEL,male,true,monthly,field_partner
3,943914,Delilah,English,Delilah is 45 years old and married with four ...,Delilah is 45 years old and married with four ...,225.0,225.0,funded,1972607.0,NaN,...,8.0,9,1,1,"#Repeat Borrower, #Vegan, #Trees, #Parent, #Wo...",Delilah,female,true,monthly,field_partner
4,149321,Hurmatoy,English,K. Hurmatoy is a happy mother of three childre...,NaN,1575.0,1575.0,funded,418838.0,NaN,...,14.0,28,1,1,NaN,Hurmatoy,female,true,monthly,field_partner


In [83]:
#Check for any duplicate loans 
len(data['LOAN_ID'].unique()) - len(data['LOAN_ID'].unique())

0

In [84]:
#Some features that could be focused on: sector,number of lenders,gender of borrows, whether or not a picture of the borrow is included, loan amount, and country
keep_cols = ['ORIGINAL_LANGUAGE','LOAN_AMOUNT','STATUS','ACTIVITY_NAME','SECTOR_NAME','COUNTRY_NAME','CURRENCY','BORROWER_GENDERS','BORROWER_PICTURED','REPAYMENT_INTERVAL','DISTRIBUTION_MODEL']
data = data[keep_cols];
data.columns

Index(['ORIGINAL_LANGUAGE', 'LOAN_AMOUNT', 'STATUS', 'ACTIVITY_NAME',
       'SECTOR_NAME', 'COUNTRY_NAME', 'CURRENCY', 'BORROWER_GENDERS',
       'BORROWER_PICTURED', 'REPAYMENT_INTERVAL', 'DISTRIBUTION_MODEL'],
      dtype='object')

In [85]:
# We want to focus on loans that are either funded or expired (meaning they did not hit their fundraising goals)
keep_rows = data['STATUS'].isin(['expired','funded'])
data = data[keep_rows]
data.STATUS.value_counts()

funded     1867176
expired      90948
Name: STATUS, dtype: int64

In [86]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1958124 entries, 0 to 1979467
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ORIGINAL_LANGUAGE   object 
 1   LOAN_AMOUNT         float64
 2   STATUS              object 
 3   ACTIVITY_NAME       object 
 4   SECTOR_NAME         object 
 5   COUNTRY_NAME        object 
 6   CURRENCY            object 
 7   BORROWER_GENDERS    object 
 8   BORROWER_PICTURED   object 
 9   REPAYMENT_INTERVAL  object 
 10  DISTRIBUTION_MODEL  object 
dtypes: float64(1), object(10)
memory usage: 179.3+ MB


In [87]:
#Create country codes & continent columns for each entry
country_list = [c.name for c in list(p.countries)]
country_codes = [c.alpha_2 for c in list(p.countries)]
country_codes.remove('TL')
data['COUNTRY_CODE'] = data['COUNTRY_NAME'].apply(lambda x: pc.country_name_to_country_alpha2(x) if x in country_list else None)
data['CONTINENT']= data['COUNTRY_CODE'].apply(lambda x:  pc.country_alpha2_to_continent_code(x) if x in country_codes else None)
data['CONTINENT'] = data['CONTINENT'].astype(str)
data['CONTINENT'].value_counts()

AS      787643
AF      528193
SA      244266
NA      229888
None    126651
OC       30636
EU       10847
Name: CONTINENT, dtype: int64

In [88]:
data.isna().sum(axis = 0)

ORIGINAL_LANGUAGE      36697
LOAN_AMOUNT                0
STATUS                     0
ACTIVITY_NAME              0
SECTOR_NAME                0
COUNTRY_NAME               0
CURRENCY                   0
BORROWER_GENDERS       36697
BORROWER_PICTURED      36697
REPAYMENT_INTERVAL         0
DISTRIBUTION_MODEL         0
COUNTRY_CODE          118261
CONTINENT                  0
dtype: int64

In [90]:
#Drop rows with any missing values
data = data.dropna()

In [62]:
data.to_csv('wrangled_loans.csv')
data.head()

,ORIGINAL_LANGUAGE,LOAN_AMOUNT,STATUS,ACTIVITY_NAME,SECTOR_NAME,COUNTRY_NAME,CURRENCY,BORROWER_GENDERS,BORROWER_PICTURED,REPAYMENT_INTERVAL,DISTRIBUTION_MODEL,COUNTRY_CODE,CONTINENT
0,Spanish,825.0,funded,Furniture Making,Manufacturing,Ecuador,USD,female,true,monthly,field_partner,EC,SA
1,English,525.0,funded,Poultry,Agriculture,Philippines,PHP,female,true,monthly,field_partner,PH,AS
2,Spanish,400.0,funded,Personal Housing Expenses,Housing,Peru,PEN,male,true,monthly,field_partner,PE,SA
3,English,225.0,funded,Farming,Agriculture,Philippines,PHP,female,true,monthly,field_partner,PH,AS
4,English,1575.0,funded,Grocery Store,Food,Tajikistan,TJS,female,true,monthly,field_partner,TJ,AS
